<a href="https://colab.research.google.com/github/garykbrixi/brain-view/blob/master/mouse_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torchvision.transforms as transforms
import torch.nn as nn
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from fastai.vision import *
from fastai.metrics import error_rate
from PIL import Image
from google_drive_downloader import GoogleDriveDownloader as gdd

import pickle

from sklearn.cross_decomposition import PLSRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from sklearn.utils.validation import check_is_fitted, check_array
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# function for torch PLSR

def svd_flip(u, v, u_based_decision=True):
    u = u.view(u.size()[0],1)
    v = v.view(v.size()[0],1)
    if u_based_decision:
        # columns of u, rows of v
        max_abs_cols = torch.argmax(torch.abs(u), axis=0)
        signs = torch.sign(u[list(max_abs_cols), range(u[0].size()[0])])
        u *= signs
        v *= signs.unsqueeze(1)
    else:
        # rows of v, columns of u
        max_abs_rows = torch.argmax(torch.abs(v), axis=1)
        signs = torch.sign(v[range(list(v.size())[0]), max_abs_rows])
        u *= signs
        v *= signs.unsqueeze(1)
    u = u.flatten()
    v = v.flatten()
    return u, v

def _nipals_twoblocks_inner_loop(X, Y, mode="A", max_iter=500, tol=1e-06,
                                 norm_y_weights=False):
    """Inner loop of the iterative NIPALS algorithm.
    Provides an alternative to the svd(X'Y); returns the first left and right
    singular vectors of X'Y.  See PLS for the meaning of the parameters.  It is
    similar to the Power method for determining the eigenvectors and
    eigenvalues of a X'Y.
    """
    for col in Y.T:
        if torch.any(torch.abs(col) > torch.finfo(torch.float).eps):

            y_score = col.detach().view(col.size())

            break

    x_weights_old = 0
    ite = 1
    X_pinv = Y_pinv = None
    eps = torch.finfo(X.dtype).eps
    if mode == "B":
        # Uses condition from scipy<1.3 in pinv2 which was changed in
        # https://github.com/scipy/scipy/pull/10067. In scipy 1.3, the
        # condition was changed to depend on the largest singular value
        X_t = X.dtype.char.lower()
        Y_t = Y.dtype.char.lower()
        factor = {'f': 1E3, 'd': 1E6}

        cond_X = factor[X_t] * eps
        cond_Y = factor[Y_t] * eps

    # Inner loop of the Wold algo.
    while True:
        # 1.1 Update u: the X weights
        if mode == "B":
            if X_pinv is None:
                # torch pinverse
                X_pinv = torch.pinverse(X, check_finite=False, cond=cond_X)
            x_weights = torch.mm(X_pinv, y_score)
        else:  # mode
            # Mode A regress each X column on y_score
            x_weights = torch.mv(X.T, y_score) / torch.dot(y_score.T, y_score)
        # If y_score only has zeros x_weights will only have zeros. In
        # this case add an epsilon to converge to a more acceptable
        # solution
        if torch.dot(x_weights.T, x_weights) < eps:
            x_weights += eps
        # 1.2 Normalize u
        x_weights /= torch.sqrt(torch.dot(x_weights.T, x_weights)) + eps
        # 1.3 Update x_score: the X latent scores
        x_score = torch.mv(X, x_weights)
        # 2.1 Update y_weights
        if mode == "B":
            if Y_pinv is None:
                # compute once pinv(Y)
                Y_pinv = torch.pinverse(Y, check_finite=False, cond=cond_Y)
            y_weights = torch.mm(Y_pinv, x_score)
        else:
            # Mode A regress each Y column on x_score
            y_weights = torch.mv(Y.T, x_score) / torch.dot(x_score.T, x_score)
        # 2.2 Normalize y_weights
        if norm_y_weights:
            y_weights /= torch.sqrt(torch.mm(y_weights.T, y_weights)) + eps
        # 2.3 Update y_score: the Y latent scores
        y_score = torch.mv(Y, y_weights) / (torch.dot(y_weights.T, y_weights) + eps)
        # y_score = np.dot(Y, y_weights) / np.dot(y_score.T, y_score) ## BUG
        x_weights_diff = x_weights - x_weights_old

        if torch.dot(x_weights_diff.T, x_weights_diff) < tol or Y.size()[1] == 1:
            break
        if ite == max_iter:
            warnings.warn('Maximum number of iterations reached',
                          ConvergenceWarning)
            break
        x_weights_old = x_weights
        ite += 1
    return x_weights, y_weights, ite

def _center_scale_xy(X, Y, scale=True):
    """ Center X, Y and scale if the scale parameter==True
    Returns
    -------
        X, Y, x_mean, y_mean, x_std, y_std
    """
    # center
    x_mean = torch.mean(X, axis=0)
    X -= x_mean
    y_mean = torch.mean(Y, axis=0)
    Y -= y_mean
    # scale
    if scale:
        x_std = torch.std(X, dim = 0)
        x_std[x_std == 0.0] = 1.0
        X = X/x_std
        y_std = torch.std(Y, dim = 0)
        y_std[y_std == 0.0] = 1.0
        Y = Y/y_std
    else:
        x_std = torch.ones(X.size()[1])
        y_std = torch.ones(Y.size()[1])
    return X, Y, x_mean, y_mean, x_std, y_std

class PLS:
    def __init__(self, n_components=2, *, scale=True,
                 deflation_mode="regression",
                 mode="A", algorithm="nipals", norm_y_weights=False,
                 max_iter=500, tol=1e-06, copy=True):
        self.n_components = n_components
        self.deflation_mode = deflation_mode
        self.mode = mode
        self.norm_y_weights = norm_y_weights
        self.scale = scale
        self.algorithm = algorithm
        self.max_iter = max_iter
        self.tol = tol
        self.copy = copy
        self.scale = scale

    def fit(self, X, Y):
        """Fit model to data.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples and
            n_features is the number of predictors.
        Y : array-like of shape (n_samples, n_targets)
            Target vectors, where n_samples is the number of samples and
            n_targets is the number of response variables.
        """
        X = X.clone().detach()
        Y = Y.clone().detach()
        if Y.ndim == 1:
            Y = Y.reshape(-1, 1)

        n = X.size()[0]
        p = X.size()[1]
        q = Y.size()[1]

        # if self.n_components < 1 or self.n_components > p:
        #     raise ValueError('Invalid number of components: %d' %
        #                      self.n_components)
        # if self.algorithm not in ("svd", "nipals"):
        #     raise ValueError("Got algorithm %s when only 'svd' "
        #                      "and 'nipals' are known" % self.algorithm)
        # if self.algorithm == "svd" and self.mode == "B":
        #     raise ValueError('Incompatible configuration: mode B is not '
        #                      'implemented with svd algorithm')
        # if self.deflation_mode not in ["canonical", "regression"]:
        #     raise ValueError('The deflation mode is unknown')
        # Scale (in place)
        X, Y, self.x_mean_, self.y_mean_, self.x_std_, self.y_std_ = (
            _center_scale_xy(X, Y, self.scale))
        # Residuals (deflated) matrices
        Xk = X
        Yk = Y

        # Results matrices
        self.x_scores_ = torch.zeros((n, self.n_components))
        self.y_scores_ = torch.zeros((n, self.n_components))
        self.x_weights_ = torch.zeros((p, self.n_components))
        self.y_weights_ = torch.zeros((q, self.n_components))
        self.x_loadings_ = torch.zeros((p, self.n_components))
        self.y_loadings_ = torch.zeros((q, self.n_components))
        self.n_iter_ = []

        # NIPALS algo: outer loop, over components
        Y_eps = torch.finfo(Yk.dtype).eps

        for k in range(self.n_components):

            if torch.all(torch.mm(Yk.T, Yk) < torch.finfo(torch.float).eps):
                # Yk constant
                warnings.warn('Y residual constant at iteration %s' % k)
                break
            # 1) weights estimation (inner loop)
            # -----------------------------------
            if self.algorithm == "nipals":
                # Replace columns that are all close to zero with zeros
                Yk_mask = torch.all(torch.abs(Yk) < 10 * Y_eps, axis=0)
                Yk[:, Yk_mask] = 0.0

                x_weights, y_weights, n_iter_ = \
                    _nipals_twoblocks_inner_loop(
                        X=Xk, Y=Yk, mode=self.mode, max_iter=self.max_iter,
                        tol=self.tol, norm_y_weights=self.norm_y_weights)
                self.n_iter_.append(n_iter_)

            elif self.algorithm == "svd":
                x_weights, y_weights = _svd_cross_product(X=Xk, Y=Yk)
            # Forces sign stability of x_weights and y_weights
            # Sign undeterminacy issue from svd if algorithm == "svd"
            # and from platform dependent computation if algorithm == 'nipals'

            x_weights, y_weights = svd_flip(x_weights, y_weights.T)
            y_weights = y_weights.T
            # columns of u, rows of v
            
            # compute scores
            x_scores = torch.mv(Xk, x_weights)

            if self.norm_y_weights:
                y_ss = 1
            else:
                y_ss = torch.dot(y_weights.T, y_weights)

            y_scores = torch.mv(Yk, y_weights) / y_ss

            # test for null variance
            if torch.dot(x_scores.T, x_scores) < torch.finfo(torch.double).eps:
                warnings.warn('X scores are null at iteration %s' % k)
                break
            # 2) Deflation (in place)
            # ----------------------
            #
            # - regress Xk's on x_score

            x_loadings = torch.mv(Xk.T, x_scores) / torch.dot(x_scores.T, x_scores)

            # - subtract rank-one approximations to obtain remainder matrix

            Xk -= x_scores[:, None] * x_loadings.T

            if self.deflation_mode == "canonical":
                # - regress Yk's on y_score, then subtract rank-one approx.
                y_loadings = (torch.mv(Yk.T, y_scores)
                              / torch.dot(y_scores.T, y_scores))
                Yk -= y_scores[:, None] * y_loadings.T
            if self.deflation_mode == "regression":
                # - regress Yk's on x_score, then subtract rank-one approx.
                y_loadings = (torch.mv(Yk.T, x_scores)
                              / torch.dot(x_scores.T, x_scores))
                Yk -= x_scores[:, None] * y_loadings.T
            # 3) Store weights, scores and loadings # Notation:

            self.x_scores_[:, k] = x_scores.view(-1)  # T
            self.y_scores_[:, k] = y_scores.view(-1)  # U
            self.x_weights_[:, k] = x_weights.view(-1)  # W
            self.y_weights_[:, k] = y_weights.view(-1)  # C
            self.x_loadings_[:, k] = x_loadings.view(-1)  # P
            self.y_loadings_[:, k] = y_loadings.view(-1)  # Q

        # Such that: X = TP' + Err and Y = UQ' + Err

        # 4) rotations from input space to transformed space (scores)
        # T = X W(P'W)^-1 = XW* (W* : p x k matrix)
        # U = Y C(Q'C)^-1 = YC* (W* : q x k matrix)
        self.x_rotations_ = torch.mm(
            self.x_weights_,
            torch.pinverse(torch.mm(self.x_loadings_.T, self.x_weights_)))
        if Y.size()[1] > 1:
            self.y_rotations_ = torch.mm(
                self.y_weights_,
                torch.pinverse(torch.mm(self.y_loadings_.T, self.y_weights_)))
        else:
            self.y_rotations_ = torch.ones(1)

        if True or self.deflation_mode == "regression":
            # Estimate regression coefficient
            # Regress Y on T
            # Y = TQ' + Err,
            # Then express in function of X
            # Y = X W(P'W)^-1Q' + Err = XB + Err
            # => B = W*Q' (p x q)

            self.coef_ = torch.mm(self.x_rotations_, self.y_loadings_.T)
            self.coef_ = self.coef_
            self.y_std_ = self.y_std_
            # self.coef_ = torch.mv(self.coef_, self.y_std_)
            self.coef_ = self.coef_[:, None] * self.y_std_
            self.coef_ = self.coef_[:,0,:]

        return self

    def transform(self, X, Y=None, copy=True):
        """Apply the dimension reduction learned on the train data.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples and
            n_features is the number of predictors.
        Y : array-like of shape (n_samples, n_targets)
            Target vectors, where n_samples is the number of samples and
            n_targets is the number of response variables.
        copy : boolean, default True
            Whether to copy X and Y, or perform in-place normalization.
        Returns
        -------
        x_scores if Y is not given, (x_scores, y_scores) otherwise.
        """
        check_is_fitted(self)
        X = check_array(X, copy=copy, dtype=FLOAT_DTYPES)
        # Normalize
        X -= self.x_mean_
        X /= self.x_std_
        # Apply rotation
        x_scores = torch.mm(X, self.x_rotations_)
        if Y is not None:
            Y = check_array(Y, ensure_2d=False, copy=copy, dtype=FLOAT_DTYPES)
            if Y.ndim == 1:
                Y = Y.reshape(-1, 1)
            Y -= self.y_mean_
            Y /= self.y_std_
            y_scores = torch.mm(Y, self.y_rotations_)
            return x_scores, y_scores

        return x_scores

    def inverse_transform(self, X):
        """Transform data back to its original space.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_components)
            New data, where n_samples is the number of samples
            and n_components is the number of pls components.
        Returns
        -------
        x_reconstructed : array-like of shape (n_samples, n_features)
        Notes
        -----
        This transformation will only be exact if n_components=n_features
        """
        check_is_fitted(self)
        X = check_array(X, dtype=FLOAT_DTYPES)
        # From pls space to original space
        X_reconstructed = torch.matmul(X, self.x_loadings_.T)

        # Denormalize
        X_reconstructed *= self.x_std_
        X_reconstructed += self.x_mean_
        return X_reconstructed

    def predict(self, X, copy=True):
        """Apply the dimension reduction learned on the train data.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples and
            n_features is the number of predictors.
        copy : boolean, default True
            Whether to copy X and Y, or perform in-place normalization.
        Notes
        -----
        This call requires the estimation of a p x q matrix, which may
        be an issue in high dimensional space.
        """
        check_is_fitted(self)
        # X = check_array(X, copy=copy, dtype=FLOAT_DTYPES)
        # Normalize

        X -= self.x_mean_
        X /= self.x_std_

        Ypred = torch.mm(X, self.coef_)
        return Ypred + self.y_mean_

    def fit_transform(self, X, y=None):
        """Learn and apply the dimension reduction on the train data.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples and
            n_features is the number of predictors.
        y : array-like of shape (n_samples, n_targets)
            Target vectors, where n_samples is the number of samples and
            n_targets is the number of response variables.
        Returns
        -------
        x_scores if Y is not given, (x_scores, y_scores) otherwise.
        """
        return self.fit(X, y).transform(X, y)

    def _more_tags(self):
        return {'poor_score': True}

In [5]:
# prepare data for analysis
PATH = "drive/My Drive/neuro140/"
infile = open(PATH+'/mouse_brain_data_sample.pkl','rb')
mouse_pickle = pickle.load(infile)
VISam = mouse_pickle['VISam']
print(VISam[118])
VISpm = mouse_pickle['VISpm']

torch_VISam = torch.tensor(VISam).float()
torch_VISpm = torch.tensor(VISpm).float()

img_array = np.load(PATH+'/stimulus_set.npy')

create_images = False
dictlist = []
img_names = []
for i in range(119):
  pngfilename = str(i)+str('.jpg')
  if (create_images):
    im = Image.fromarray(img_array[i])
    im.save(PATH + 'images/' + pngfilename)
  row = {'ImageName': pngfilename, 'ImageIndex': i}
  img_names.append(pngfilename)
  dictlist.append(row)

[0.120102 0.020811 0.       0.018055 ... 0.008298 0.004659 0.       0.      ]


In [0]:
# get activations from images per model

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

from tqdm import tqdm_notebook as tqdm

from fastai.torch_core import flatten_model

preprocessing = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class SaveFeatures():
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output.clone().detach().requires_grad_(True).cuda()
    def close(self):
        self.hook.remove()
        
def get_layer_names(layers):
    layer_names = []
    for layer in layers:
        layer_name = str(layer).split('(')[0]
        layer_names.append(layer_name + '-' + str(sum(layer_name in string for string in layer_names) + 1))
    return layer_names

def get_activations(model, img, layers, target_layer):
    img_tensor = preprocessing(img).unsqueeze(0).cuda()
    activations = SaveFeatures(layers[target_layer])
    model(img_tensor)
    activations.close()
    return activations.features.detach().cpu().numpy().squeeze()
##################################################################

res = False
alex = False
mobile = False
inception = False

if res:
  model = models.resnet18(pretrained=True)
  layers = flatten_model(model)
  layer_names = get_layer_names(layers)
  print(len(layer_names))
  print(layer_names)

  path = Path(PATH + 'images/')

  layer_wise_activation = False
  if(layer_wise_activation):
    classifier_rdms = {}
    for layer_index, layer in enumerate(tqdm(layers[:-2])):
        image_activations = []
        for image in img_names:
            image_array = Image.open(path/image)
            # print(get_activations(model, image_array, layers, layer_index).flatten())
            image_activations.append(get_activations(model, image_array, layers, layer_index).flatten())
        layer_features = np.stack(image_activations)
    #save file
    with open(PATH + 'classifier_dict.pickle', 'wb') as f:
      pickle.dump(classifier_rdms, f)

  image_wise_activations = False
  if(image_wise_activations):
    image_activations = []
    for image in img_names:
      image_array = Image.open(path/image)
      layer_activations = []
      for layer_index, layer in enumerate(tqdm(layers[:-2])):
        layer_activations.append(get_activations(model, image_array, layers, layer_index))
      image_activations.append(layer_activations)
    #save file
    x = np.array(image_activations)
    np.save(str(PATH + 'res18_image_activations.npy'), x, allow_pickle=True)

if(alex):
  model = models.alexnet(pretrained=True)
  layers = flatten_model(model)
  layer_names = get_layer_names(layers)
  image_activations = []
  for image in img_names:
    image_array = Image.open(path/image)
    layer_activations = []
    for layer_index, layer in enumerate(tqdm(layers[:-2])):
      layer_activations.append(get_activations(model, image_array, layers, layer_index))
    image_activations.append(layer_activations)
  x = np.array(image_activations)
  np.save(str(PATH + 'alex_image_activations.npy'), x, allow_pickle=True)

if(mobile):
  model = models.mobilenet_v2(pretrained=True)
  layers = flatten_model(model)
  layer_names = get_layer_names(layers)
  image_activations = []
  for image in img_names:
    image_array = Image.open(path/image)
    layer_activations = []
    for layer_index, layer in enumerate(tqdm(layers[:-2])):
      layer_activations.append(get_activations(model, image_array, layers, layer_index))
    image_activations.append(layer_activations)
  x = np.array(image_activations)
  np.save(str(PATH + 'mobile_image_activations.npy'), x, allow_pickle=True)

inception_image_wise_activations = False
if(inception_image_wise_activations):
  model = models.inception_v3()(pretrained=True)
  layers = flatten_model(model)
  layer_names = get_layer_names(layers)
  image_activations = []
  for image in img_names:
    image_array = Image.open(path/image)
    layer_activations = []
    for layer_index, layer in enumerate(tqdm(layers[:-2])):
      layer_activations.append(get_activations(model, image_array, layers, layer_index))
    image_activations.append(layer_activations)
  x = np.array(image_activations)
  np.save(str(PATH + 'inception_image_activations.npy'), x, allow_pickle=True)

In [0]:
#regress from NN layer to predict the neuron
torch.set_default_tensor_type(torch.cuda.FloatTensor)

def np_12split_PLSR (datasets, layer, savename):
  kfold_splits = 12
  dataset = np.transpose(datasets)
  list_sk = []
  corr_list = []
  for i in range(len(dataset)):
    neuron = dataset[i].reshape(-1, 1)
    predictions = np.zeros((119,1))
    for split in range(kfold_splits):
      test_ind = list(range(split*10, min(split*10+10, 119)))
      train_ind = list(set(list(range(0, 119)))^set(test_ind))
      Y_test = neuron[test_ind, :]
      Y_train = neuron[train_ind, :]
      x = layer
      X_test = x[test_ind]
      X_train = x[train_ind]
      clf = PLSRegression(n_components=25).fit(X_train, Y_train)
      predictions[test_ind] = clf.predict(X_test)
    list_sk.append(predictions)
    corr_list.append(np.correlate(predictions.reshape(1,-1)[0],neuron.reshape(1,-1)[0]))
  np.save(PATH+savename+'_list_sk.npy', np.array(list_sk))
  np.save(PATH+savename+'_corr.npy', np.array(corr_list))

def torch_12split_PLSR (datasets, layer, savename):
  #pytorch version!
  kfold_splits = 12
  dataset = datasets.T
  list_sk = []
  corr_list = []
  for i in range(len(dataset)):
    neuron = dataset[i].reshape(-1, 1)
    predictions = np.zeros((119,1))
    for split in range(kfold_splits):
      test_ind = list(range(split*10, min(split*10+10, 119)))
      train_ind = list(set(list(range(0, 119)))^set(test_ind))
      Y_test = neuron[test_ind, :]
      Y_train = neuron[train_ind, :]
      x = layer
      X_test = x[test_ind]
      X_train = x[train_ind]
      clf = PLS(n_components=25).fit(X_train, Y_train)
      predictions[test_ind] = clf.predict(X_test).cpu()
    list_sk.append(predictions)
    corr_list.append(np.correlate(predictions.reshape(1,-1)[0],np.array(neuron.cpu()).reshape(1,-1)[0]))
  np.save(PATH+'torch_'+savename+'_list_sk.npy', np.array(list_sk))
  np.save(PATH+"torch"+savename+"_corr.npy", np.array(corr_list))

if res:
  image_activations = np.load(str(PATH+'image_activations.npy'), allow_pickle=True)

  layer_0 = np.zeros((119, 200704))
  torch_layer_0 = torch.zeros((119, 200704)).cuda()
  layer_4 = np.zeros((119, 200704))
  torch_layer_4 = torch.zeros((119, 200704)).cuda()
  layer_48 = np.zeros((119, 25088))
  torch_layer_48 = torch.zeros((119, 25088)).cuda()
  for i in range(len(image_activations)):
    layer_0[i] = image_activations[i][0].flatten()
    torch_layer_0[i] = torch.from_numpy(image_activations[i][0]).flatten()
    layer_4[i] = image_activations[i][4].flatten()
    torch_layer_4[i] = torch.from_numpy(image_activations[i][4]).flatten()
    layer_48[i] = image_activations[i][48].flatten()
    torch_layer_48[i] = torch.from_numpy(image_activations[i][48]).flatten()

if alex:
  image_activations = np.load(str(PATH+'alex_image_activations.npy'), allow_pickle=True)
  
  alex_0 = np.zeros((119, 193600))
  torch_alex_0 = torch.zeros((119, 193600)).cuda()
  alex_3 = np.zeros((119, 139968))
  torch_alex_3 = torch.zeros((119, 139968)).cuda()
  alex_6 = np.zeros((119, 64896))
  torch_alex_6 = torch.zeros((119, 64896)).cuda()
  alex_8 = np.zeros((119, 43264))
  torch_alex_8 = torch.zeros((119, 43264)).cuda()
  alex_10 = np.zeros((119, 43264))
  torch_alex_10 = torch.zeros((119, 43264)).cuda()
  for i in range(len(image_activations_alex)):
    alex_0[i] = image_activations[i][0].flatten()
    torch_alex_0[i] = torch.from_numpy(image_activations[i][0]).flatten().float()
    alex_3[i] = image_activations[i][3].flatten()
    torch_alex_3[i] = torch.from_numpy(image_activations[i][3]).flatten().float()
    alex_6[i] = image_activations[i][6].flatten()
    torch_alex_6[i] = torch.from_numpy(image_activations[i][6]).flatten().float()
    alex_8[i] = image_activations[i][8].flatten()
    torch_alex_8[i] = torch.from_numpy(image_activations[i][8]).flatten().float()
    alex_10[i] = image_activations[i][10].flatten()
    torch_alex_10[i] = torch.from_numpy(image_activations[i][10]).flatten().float()

if mobile:
  image_activations_alex = np.load(str(PATH+'mobile_image_activations.npy'), allow_pickle=True)
  
  mobile_0 = np.zeros((119, 802816))
  torch_mobile_0 = torch.zeros((119, 802816)).cuda()
  # mobile_3 = np.zeros((119, 200704))
  # torch_mobile_3 = torch.zeros((119, 200704)).cuda()
  # mobile_6 = np.zeros((119, 200704))
  # torch_mobile_6 = torch.zeros((119, 200704)).cuda()
  mobile_8 = np.zeros((119, 200704))
  torch_mobile_8 = torch.zeros((119, 200704)).cuda()
  for i in range(len(image_activations_alex)):
    mobile_0[i] = image_activations_res[i][0].flatten()
    torch_mobile_0[i] = torch.from_numpy(image_activations_res[i][0]).flatten().float()
    mobike_8[i] = image_activations_res[i][8].flatten()
    torch_mobile_8[i] = torch.from_numpy(image_activations_res[i][8]).flatten().float()

if res:
  np_12split_PLSR(VISam, layer_4, 'res18_4_VISam')

  np_12split_PLSR(VISam, layer_48, 'res18_48_VISam')

  np_12split_PLSR(VISam, layer_4, 'res18_4_VISpm')

  np_12split_PLSR(VISam, layer_4, 'res18_48_VISpm')

  torch_12split_PLSR(torch_VISam, torch_layer_4, 'torch_res18_4_VISam')

  torch_12split_PLSR(torch_VISam, torch_layer_48, 'torch_res18_48_VISam')

  torch_12split_PLSR(torch_VISpm, torch_layer_4, 'torch_res18_4_VISpm')

  torch_12split_PLSR(torch_VISpm, torch_layer_48, 'torch_res18_48_VISpm')

if alex:
  # np_12split_PLSR(VISam, alex_0, 'alex_0_VISam')

  # np_12split_PLSR(VISam, alex_3, 'alex_3_VISam')

  # np_12split_PLSR(VISam, alex_10, 'alex_10_VISam')

  # np_12split_PLSR(VISpm, alex_0, 'alex_0_VISpm')

  # np_12split_PLSR(VIpam, alex_3, 'alex_3_VISpm')

  # np_12split_PLSR(VISpm, alex_10, 'alex_10_VISpm')

  torch_12split_PLSR(torch_VISam, torch_alex_0, 'torch_alex_0_VISam')

  torch_12split_PLSR(VISam, torch_alex_3, 'torch_alex_3_VISam')

  torch_12split_PLSR(torch_VISam, torch_alex_10, 'alex_10_VISam')

  torch_12split_PLSR(torch_VISpm, torch_alex_0, 'alex_0_VISpm')

  torch_12split_PLSR(torch_VISpm, torch_alex_3, 'torch_alex_3_VISpm')

  torch_12split_PLSR(torch_VISpm, torch_alex_10, 'alex_10_VISpm')

if mobile:
  np_12split_PLSR(VISam, mobile_0, 'mobile_0_VISam')

  np_12split_PLSR(VISam, mobile_8, 'mobile_8_VISam')

  np_12split_PLSR(VISpm, mobile_0, 'mobile_0_VISpm')

  np_12split_PLSR(VISpm, mobile_8, 'mobile_8_VISpm')


In [15]:
res18_4_corr = np.load(PATH+'torch_res18_3_VISam_corr.npy')
print('res18 4 sam')
print(np.average(res18_4_corr))
res18_corr = np.load(PATH+'res18_48_VISam_corr.npy')
print('res18 48 sam')
print(np.average(res18_corr))
res18_4_corr = np.load(PATH+'torchtorch_res18_4_VISpm_corr.npy')
print('res18 4 spm')
print(np.average(res18_4_corr))
res18_48_corr = np.load(PATH+'res18_48_VISpm_corr.npy')
print('res18 48 spm')
print(np.average(res18_48_corr))

corr = np.load(PATH+'alex_0_VISam_corr.npy')
print('alex 0 sam')
print(np.average(corr))
corr = np.load(PATH+'torchalex_10_VISam_corr.npy')
print('alex 10 sam')
print(np.average(corr))

corr = np.load(PATH+'torchalex_0_VISpm_corr.npy')
print('alex 0 spm')
print(np.average(corr))
corr = np.load(PATH+'torchalex_10_VISpm_corr.npy')
print('alex 10 spm')
print(np.average(corr))

corr_dict = {}

res18 4 sam
0.21279011529734965
res18 48 sam
0.18767793624548623
res18 4 spm
0.16978219607235842
res18 48 spm
0.14796026100829673
alex 0 sam
0.21279016089904032
alex 10 sam
0.24477178764847266
alex 0 spm
0.10018296656487818
alex 10 spm
0.18821280066774534


In [0]:
#benchmarking: PLSR
#using layer 4 of resnet 18, VISam neurons, all images
# x dim : 119, 

if (res):
  layer_4 = layer_4.astype(np.float32)
  VISam = VISam.astype(np.float32)

  # print(layer_4.shape)
  # print(torch_layer_4.size())

  # for i in range(11):
  #   # index = list(range(i*10 + 9))
  #   index = list(range(119))
  #   #torch version:
    # %timeit PLS(n_components=25).fit(torch_layer_4[index], torch_VISam[index, :])
  #   #scikit
  #   %timeit PLSRegression(n_components=25).fit(layer_4[index], VISam[index, :])

  # check equality
  equality_check = True
  if equality_check:
    index = list(range(118))
    train = 119
    print(torch_layer_4)
    modl = PLS(n_components=25).fit(torch_layer_4[index], torch_VISam[index, :])
    torch_ans = modl.predict(torch_layer_4[train])
    modl = PLSRegression(n_components=25).fit(layer_4[index], VISam[index, :])
    sk_ans = modl.predict(layer_4[train])
    print(torch_ans)
    print(sk_ans)
    # assert torch_ans == sk_ans

In [0]:
def cov(x, rowvar = False):
  if not rowvar:
    x = x.t()
  f = 1.0 / (x.size(1) - 1)
  x -= torch.mean(x, dim=1, keepdim=True)
  xt = x.t()
  c = f * x.matmul(xt).squeeze()
  return c


def PCA(data, dims = 2):
  """
  based on numpy function:
  https://stackoverflow.com/questions/13224362/principal-component-analysis-pca-in-python
  """
  n = data.shape
  mx = torch.mean(data, dim=0)

  x = data - mx
  R = cov(x, False)

  evals, evecs = torch.symeig(R_t, eigenvectors = True, upper = True)
  
  idx = torch.argsort(evals, descending = True)
  evecs = evecs[:,idx]
  evals = evals[idx]

  evecs = evecs[:, :dims]

  return torch.mm(evecs.T, data.T).T, evals, evecs


from sklearn.decomposition import PCA as PCA_sk

pca = PCA_sk(n_components=5)
pca.fit(VISam)
Sam_c = pca.components_
print(Sam_c)

pca = PCA_sk(n_components=5)
pca.fit(VIspm)
Spm_c = pca.components_
print(Spm_c)